In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/jre_elon_musk.csv", index_col=False)
df.head()

,Timestamp,Speaker,Text
0,[00:00:00],Joe Rogan,"Ah, ha, ha, ha. Four, three, two, one, boom. T..."
1,[00:00:09],Elon Musk,You're welcome.
2,[00:00:10],Joe Rogan,It's very good to meet you.
3,[00:00:11],Elon Musk,Nice to meet you too.
4,[00:00:12],Joe Rogan,And thanks for not lighting this place on fire.


In [4]:
import spacy

nlp = spacy.load("en_core_web_trf")

In [5]:
for i in range(11): #(df.shape[0]):
    txt = df.iloc[i, 2]
    doc = nlp(txt)
    print(doc)

c:\NLPSpacey\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Ah, ha, ha, ha. Four, three, two, one, boom. Thank you. Thanks for doing this, man. Really appreciate it.
You're welcome.
It's very good to meet you.
Nice to meet you too.
And thanks for not lighting this place on fire.
You're welcome. That's coming later.
How does one, just in the middle of doing all the things you do, create cars, rockets, all the stuff you're doing,constantly innovating, decide to just make a flamethrower? Where do you have the time for that?
Well, the flame, we didn't put a lot of time into the flamethrower. This was an off-the-cuff thing. It's sort of a hobbycompany called the Boring Company, which started out as a joke, and we decided to make a real, and dig a tunnelunder LA. And then, other people asked us to dig tunnels. And so, we said yes in a few cases.
Now, who-
And then, we have a merchandise section that only has one piece of merchandise at a time. And we started off witha cap. And there was only one thing on, which is BoringCompany.com/hat. That's it. An

In [6]:
# Create a function to convert timestamp into seconds
import re

def convert_timestamp_into_seconds(df):
    df_timestamp = pd.DataFrame({'TimeStamp': []})
    for i in range(df.shape[0]):
        x = df.iloc[i,0].split(':')
        a = (int(x[0][1:])*60 + int(x[1])) * 60 + int(x[2][:-1])
        df_timestamp = df_timestamp.append({'TimeStamp': a}, ignore_index=True)

    df['Timestamp'] = df_timestamp
    return df

In [7]:
df2 = convert_timestamp_into_seconds(df)
df2

,Timestamp,Speaker,Text
0,0.0,Joe Rogan,"Ah, ha, ha, ha. Four, three, two, one, boom. T..."
1,9.0,Elon Musk,You're welcome.
2,10.0,Joe Rogan,It's very good to meet you.
3,11.0,Elon Musk,Nice to meet you too.
4,12.0,Joe Rogan,And thanks for not lighting this place on fire.
...,...,...,...
1826,9401.0,Joe Rogan,"I believe it's true too. So, thank you."
1827,9403.0,Elon Musk,You're welcome.
1828,9404.0,Joe Rogan,"All you assholes out there, be nice. Be nice, ..."
1829,9410.0,Elon Musk,"All right, thank you."


In [16]:
# Add a column with the seconds lasted by the text in the row
# for example: the first row lasts 9 seconds, since Elon Musk
# answer at the second 9 (Hint: use shift with period - 1).
# In case of negative numbers, just make them 1 (minimum interval lenght is 1 second)
import numpy as np
# def max1(x):
#     return np.maximum(1,x)

def add_interval_col(df):
    df_interval = pd.DataFrame({'Interval': []})
    for i in range(df.shape[0]-1):
        x = int(df.iloc[i+1, 0]) - int(df.iloc[i, 0])
        if x == 0:
            x += 1
        df_interval = df_interval.append({'Interval': x}, ignore_index=True)

    dataframe = pd.concat([df, df_interval], axis=1)
    dataframe = dataframe.replace(np.nan, 1)
    return dataframe

df3 = add_interval_col(df2)
df3

,Timestamp,Speaker,Text,Interval
0,0.0,Joe Rogan,"Ah, ha, ha, ha. Four, three, two, one, boom. T...",9.0
1,9.0,Elon Musk,You're welcome.,1.0
2,10.0,Joe Rogan,It's very good to meet you.,1.0
3,11.0,Elon Musk,Nice to meet you too.,1.0
4,12.0,Joe Rogan,And thanks for not lighting this place on fire.,1.0
...,...,...,...,...
1826,9401.0,Joe Rogan,"I believe it's true too. So, thank you.",2.0
1827,9403.0,Elon Musk,You're welcome.,1.0
1828,9404.0,Joe Rogan,"All you assholes out there, be nice. Be nice, ...",6.0
1829,9410.0,Elon Musk,"All right, thank you.",1.0


In [9]:
df.Speaker.unique()

array(['Joe Rogan', 'Elon Musk', 'Jaime'], dtype=object)

In [10]:
sum_intevals = df3.groupby('Speaker')['Interval'].sum()

# Total seconds spoken by Joe Rogan
joe_seconds = sum_intevals[2]

# Total seconds spoken by Elon Musk
elon_seconds = sum_intevals[0]

# Total seconds spoken by Jaime
jaime_seconds = sum_intevals[1]

sum_intevals

Speaker
Elon Musk    5075.0
Jaime          45.0
Joe Rogan    4637.0
Name: Interval, dtype: float64

In [11]:

elon_seconds, jaime_seconds, joe_seconds

(5075.0, 45.0, 4637.0)

In [12]:
elon = len(df3.query("Speaker == 'Elon Musk'"))
jaime = len(df3.query("Speaker == 'Jaime'")) 
joe = len(df3.query("Speaker == 'Joe Rogan'"))

elon, jaime, joe

(909, 17, 905)

In [13]:
elon_seconds/elon, jaime_seconds/jaime, joe_seconds/joe

(5.583058305830583, 2.6470588235294117, 5.1237569060773485)

Who speaks faster?

Tokenize the text, preprocess the tokens so that you have only words (excluding punctuation) and compute the velocity of each speaker as: number of words per interval / length of interval . Store the result in a column named Velocity and compute the average for each speaker.

In [46]:
# Preprocess the data
# nlp = spacy.load("en_core_web_sm")

# Create a function to remove punctuation & to remove stop words from text
# def remove_punctuation(df):
#     df_nopunct = pd.DataFrame({'NoPunctSW': []})
#     for i in range(5):#df.shape[0]):
#         txt = df.iloc[i, 2]
#         doc = nlp(txt)
#         x = [' '.join(token.text) for token in doc if not (token.is_punct & token.is_stop)]
#         # broken, gives this   [Y o u, ' r e, w e l c o m e]
#         df_nopunct = df_nopunct.append({'NoPunctSW': x}, ignore_index=True)
#     dataframe = pd.concat([df, df_nopunct], axis=1)
#     return dataframe

# df4 = remove_punctuation(df3)
# df4

In [50]:
df3['NoPunctuation'] = df3.Text.apply(lambda text: " ".join(token.text for token in nlp(text) if not token.is_punct))
# df3['NoPunctuation'] = df3['Text'].str.replace('[^\w\s]','')
df3['NoStopWords'] = df3.NoPunctuation.apply(lambda text: " ".join(token.text for token in nlp(text) if not token.is_stop))
df3.head()

c:\NLPSpacey\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


,Timestamp,Speaker,Text,Interval,NoPunctuation,NoStopWords
0,0.0,Joe Rogan,"Ah, ha, ha, ha. Four, three, two, one, boom. T...",9.0,Ah ha ha ha Four three two one boom Thank you ...,Ah ha ha ha boom Thank Thanks man appreciate
1,9.0,Elon Musk,You're welcome.,1.0,You 're welcome,welcome
2,10.0,Joe Rogan,It's very good to meet you.,1.0,It 's very good to meet you,good meet
3,11.0,Elon Musk,Nice to meet you too.,1.0,Nice to meet you too,Nice meet
4,12.0,Joe Rogan,And thanks for not lighting this place on fire.,1.0,And thanks for not lighting this place on fire,thanks lighting place fire


In [51]:
# Create a function to count the non punctuation token of a text

def count_tokens(df):
    df_ntokens = pd.DataFrame({'n_tokens': []})
    for i in range(df.shape[0]):
        txt = df.iloc[i, 4]
        doc = nlp(txt)
        x = len(doc)
        df_ntokens = df_ntokens.append({'n_tokens': x}, ignore_index=True)
    dataframe = pd.concat([df, df_ntokens], axis=1)
    return dataframe

df_final = count_tokens(df3)

c:\NLPSpacey\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [52]:
df_final.head()

,Timestamp,Speaker,Text,Interval,NoPunctuation,NoStopWords,n_tokens
0,0.0,Joe Rogan,"Ah, ha, ha, ha. Four, three, two, one, boom. T...",9.0,Ah ha ha ha Four three two one boom Thank you ...,Ah ha ha ha boom Thank Thanks man appreciate,19.0
1,9.0,Elon Musk,You're welcome.,1.0,You 're welcome,welcome,3.0
2,10.0,Joe Rogan,It's very good to meet you.,1.0,It 's very good to meet you,good meet,7.0
3,11.0,Elon Musk,Nice to meet you too.,1.0,Nice to meet you too,Nice meet,5.0
4,12.0,Joe Rogan,And thanks for not lighting this place on fire.,1.0,And thanks for not lighting this place on fire,thanks lighting place fire,9.0


In [53]:
# df_final.to_csv('data/jre_elon_musk_processed.csv', index=False)

In [ ]:
df

,Timestamp,Speaker,Text,Interval,TextNoPunct,n_tokens
0,0,Joe Rogan,"Ah, ha, ha, ha. Four, three, two, one, boom. T...",9.0,Ah ha ha ha Four three two one boom Thank you ...,19
1,9,Elon Musk,You're welcome.,1.0,You 're welcome,3
2,10,Joe Rogan,It's very good to meet you.,1.0,It 's very good to meet you,7
3,11,Elon Musk,Nice to meet you too.,1.0,Nice to meet you too,5
4,12,Joe Rogan,And thanks for not lighting this place on fire.,1.0,And thanks for not lighting this place on fire,9
...,...,...,...,...,...,...
1826,2921,Joe Rogan,"I believe it's true too. So, thank you.",2.0,I believe it 's true too So thank you,9
1827,2923,Elon Musk,You're welcome.,1.0,You 're welcome,3
1828,2924,Joe Rogan,"All you assholes out there, be nice. Be nice, ...",6.0,All you assholes out there be nice Be nice bit...,18
1829,2930,Elon Musk,"All right, thank you.",1.0,All right thank you,4


In [55]:
# Compute the velocity and store it in a new column

df_final["Velocity"] = df_final['n_tokens'] / df_final['Interval']
df_final.head()

,Timestamp,Speaker,Text,Interval,NoPunctuation,NoStopWords,n_tokens,Velocity
0,0.0,Joe Rogan,"Ah, ha, ha, ha. Four, three, two, one, boom. T...",9.0,Ah ha ha ha Four three two one boom Thank you ...,Ah ha ha ha boom Thank Thanks man appreciate,19.0,2.111111
1,9.0,Elon Musk,You're welcome.,1.0,You 're welcome,welcome,3.0,3.000000
2,10.0,Joe Rogan,It's very good to meet you.,1.0,It 's very good to meet you,good meet,7.0,7.000000
3,11.0,Elon Musk,Nice to meet you too.,1.0,Nice to meet you too,Nice meet,5.0,5.000000
4,12.0,Joe Rogan,And thanks for not lighting this place on fire.,1.0,And thanks for not lighting this place on fire,thanks lighting place fire,9.0,9.000000


,Timestamp,Speaker,Text,Interval,TextNoPunct,n_tokens,Velocity
0,0,Joe Rogan,"Ah, ha, ha, ha. Four, three, two, one, boom. T...",9.0,Ah ha ha ha Four three two one boom Thank you ...,19,2.111111
1,9,Elon Musk,You're welcome.,1.0,You 're welcome,3,3.000000
2,10,Joe Rogan,It's very good to meet you.,1.0,It 's very good to meet you,7,7.000000
3,11,Elon Musk,Nice to meet you too.,1.0,Nice to meet you too,5,5.000000
4,12,Joe Rogan,And thanks for not lighting this place on fire.,1.0,And thanks for not lighting this place on fire,9,9.000000
...,...,...,...,...,...,...,...
1826,2921,Joe Rogan,"I believe it's true too. So, thank you.",2.0,I believe it 's true too So thank you,9,4.500000
1827,2923,Elon Musk,You're welcome.,1.0,You 're welcome,3,3.000000
1828,2924,Joe Rogan,"All you assholes out there, be nice. Be nice, ...",6.0,All you assholes out there be nice Be nice bit...,18,3.000000
1829,2930,Elon Musk,"All right, thank you.",1.0,All right thank you,4,4.000000


In [56]:
df_final.groupby('Speaker')['Velocity'].mean()

Speaker
Elon Musk    2.885230
Jaime        3.630031
Joe Rogan    3.060527
Name: Velocity, dtype: float64

In [ ]:
# Inspect the avg velocity of each speaker


Speaker
Elon Musk    2.882109
Jaime        3.597523
Joe Rogan    3.046772
Name: Velocity, dtype: float64

After all, Elon was high...

![image](https://bsmedia.business-standard.com/_media/bs/img/article/2018-09/09/full/1536463138-6668.jpg)